In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm

### **Trade quantities**
* Sell $N_p = 1000$ put contracts
* Buy $N_c$ call contracts (paid for using put premium)
* Cash leftover: $C_0$ from strategy

### **Underlying**
* SPY price:
  * At initiation: $S_0$
  * One week before expiry: $S_t$

### **Option values**
* Put:
  * At initiation: $V_{p,0}$
  * At time $t$: $V_{p,t}$
* Call:
  * At initiation: $V_{c,0}$
  * At time $t$: $V_{c,t}$

---

## **Premium-Financing Constraint**

Put-sale premium finances calls plus leftover cash:

$$1000 \cdot V_{p,0} = N_c \cdot V_{c,0} + C_0$$

Hence:

$$N_c = \frac{1000 \cdot V_{p,0} - C_0}{V_{c,0}}$$

---

## **P&L One Week Before Expiry**

Total mark-to-market P&L at time $t$:

$$\text{PNL}_t = \underbrace{1000 \cdot (V_{p,0} - V_{p,t})}_{\text{Short Put P\&L}} + \underbrace{N_c \cdot (V_{c,t} - V_{c,0})}_{\text{Long Call P\&L}} + \underbrace{C_0(e^{rt} - 1)}_{\text{Interest on Cash}}$$

Substitute $N_c = \frac{1000 \cdot V_{p,0} - C_0}{V_{c,0}}$:

$$\boxed{\text{PNL}_t = 1000 \cdot (V_{p,0} - V_{p,t}) + \frac{1000 \cdot V_{p,0} - C_0}{V_{c,0}} \cdot (V_{c,t} - V_{c,0}) + C_0(e^{rt} - 1)}$$

---

## **Interpretation**

* If the 95% put decays or SPY rises → **positive** put P&L
* If the 105% call gains value → **positive** call P&L
* Cash earns risk-free interest at rate $r$ over time $t$
* Combined structure ≈ **short vol + long convexity financed using premium + interest on residual cash**

In [2]:
class OptionsPricingModel:
    """Black-Scholes-Merton model for SPY options pricing"""
    
    def __init__(self, spot_price, risk_free_rate, dividend_yield):
        self.S0 = spot_price
        self.r = risk_free_rate
        self.q = dividend_yield  # SPY dividend yield
        
    def black_scholes_price(self, K, T, sigma, option_type='call'):
        """
        Calculate Black-Scholes price for European options
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Implied volatility
        option_type: 'call' or 'put'
        """
        d1 = (np.log(self.S0 / K) + (self.r - self.q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        
        if option_type == 'call':
            price = self.S0 * np.exp(-self.q * T) * norm.cdf(d1) - K * np.exp(-self.r * T) * norm.cdf(d2)
        else:  # put
            price = K * np.exp(-self.r * T) * norm.cdf(-d2) - self.S0 * np.exp(-self.q * T) * norm.cdf(-d1)
        
        return price
    
    
    def crr_binomial_price(self, K, T, sigma, option_type='call', steps=1000, american=True):
        """
        Calculate option price using Cox-Ross-Rubinstein binomial tree
        
        Parameters:
        K: Strike price
        T: Time to maturity (in years)
        sigma: Volatility
        option_type: 'call' or 'put'
        steps: Number of time steps in the binomial tree
        american: True for American options, False for European
        """
        dt = T / steps  # Time increment
        
        # CRR parameters
        u = np.exp(sigma * np.sqrt(dt))  # Up factor
        d = 1 / u  # Down factor
        p = (np.exp((self.r - self.q) * dt) - d) / (u - d)  # Risk-neutral probability
        discount = np.exp(-self.r * dt)  # Discount factor per step
        
        # Initialize asset prices at maturity
        ST = np.zeros(steps + 1)
        for i in range(steps + 1):
            ST[i] = self.S0 * (u ** (steps - i)) * (d ** i)
        
        # Initialize option values at maturity
        if option_type == 'call':
            option_values = np.maximum(ST - K, 0)
        else:  # put
            option_values = np.maximum(K - ST, 0)
        
        # Backward induction through the tree
        for step in range(steps - 1, -1, -1):
            for i in range(step + 1):
                # Current stock price at this node
                S = self.S0 * (u ** (step - i)) * (d ** i)
                
                # Continuation value (discounted expected value)
                continuation_value = discount * (p * option_values[i] + (1 - p) * option_values[i + 1])
                
                # Early exercise value
                if option_type == 'call':
                    exercise_value = max(S - K, 0)
                else:  # put
                    exercise_value = max(K - S, 0)
                
                # For American options, take the maximum
                if american:
                    option_values[i] = max(continuation_value, exercise_value)
                else:
                    option_values[i] = continuation_value
        
        return option_values[0]

In [3]:
from dataclasses import dataclass

@dataclass
class TradeInputs:
    # Market / model inputs
    spot_price: float
    risk_free_rate: float
    dividend_yield: float
    time_to_maturity: float  # in years

    # Put (SELL) 95% moneyness
    put_strike: float
    put_contracts: int
    put_iv: float

    # Call (BUY) 105% moneyness
    call_strike: float
    call_iv: float

    # Binomial settings
    binomial_steps: int
    american: bool

    # Which model to use for trade sizing: "bs" or "binomial"
    pricing_model: str  # e.g. "bs" or "binomial"


class BullishRiskReversalTradeAnalyzer:
    """
    Encapsulates the full pricing + report logic
    for the short put / long call trade.
    """

    def __init__(self, inputs: TradeInputs):
        self.inputs = inputs
        # Initialize the underlying pricing model
        self.model = OptionsPricingModel(
            spot_price=inputs.spot_price,
            risk_free_rate=inputs.risk_free_rate,
            dividend_yield=inputs.dividend_yield,
        )

    def compute(self) -> dict:
        """
        Run all pricing steps and return a dictionary of results
        (no printing here – just pure calculation).
        """
        i = self.inputs

        # === Step 4: Price the options (Black–Scholes) ===
        put_price_bs = self.model.black_scholes_price(
            K=i.put_strike,
            T=i.time_to_maturity,
            sigma=i.put_iv,
            option_type='put'
        )

        call_price_bs = self.model.black_scholes_price(
            K=i.call_strike,
            T=i.time_to_maturity,
            sigma=i.call_iv,
            option_type='call'
        )

        # === Step 4b: Price the options (CRR Binomial) ===
        put_price_binom = self.model.crr_binomial_price(
            K=i.put_strike,
            T=i.time_to_maturity,
            sigma=i.put_iv,
            option_type='put',
            steps=i.binomial_steps,
            american=i.american
        )

        call_price_binom = self.model.crr_binomial_price(
            K=i.call_strike,
            T=i.time_to_maturity,
            sigma=i.call_iv,
            option_type='call',
            steps=i.binomial_steps,
            american=i.american
        )

        # === Choose which model to use for trade sizing ===
        pm = i.pricing_model.lower()
        if pm == "bs":
            put_price_for_trade = put_price_bs
            call_price_for_trade = call_price_bs
        elif pm == "binomial":
            put_price_for_trade = put_price_binom
            call_price_for_trade = call_price_binom
        else:
            raise ValueError(f"Unknown pricing_model: {i.pricing_model}. Use 'bs' or 'binomial'.")

        # === Step 5: Trade economics (using chosen model for sizing) ===

        # Premium collected from selling puts
        premium_collected = put_price_for_trade * i.put_contracts * 100

        # Max number of call contracts you can afford with that premium
        if call_price_for_trade > 0:
            call_contracts = int(premium_collected // (call_price_for_trade * 100))
        else:
            call_contracts = 0

        # Premium paid for calls using that size
        premium_paid = call_price_for_trade * call_contracts * 100

        net_premium = premium_collected - premium_paid

        return {
            "spot_price": i.spot_price,
            "risk_free_rate": i.risk_free_rate,
            "dividend_yield": i.dividend_yield,
            "time_to_maturity": i.time_to_maturity,
            "binomial_steps": i.binomial_steps,

            "pricing_model": pm,

            "put_strike": i.put_strike,
            "put_iv": i.put_iv,
            "put_contracts": i.put_contracts,
            "put_price_bs": put_price_bs,
            "put_price_binom": put_price_binom,

            "call_strike": i.call_strike,
            "call_iv": i.call_iv,
            "call_price_bs": call_price_bs,
            "call_price_binom": call_price_binom,

            # Trade economics (based on chosen model)
            "premium_collected": premium_collected,
            "call_contracts": call_contracts,
            "premium_paid": premium_paid,
            "net_premium": net_premium,
        }

    def print_report(self) -> None:
        """
        Pretty-print the same info the script printed,
        using the results from compute().
        """
        r = self.compute()

        print("=== TRADE SETUP ===")
        print(f"Spot Price: ${r['spot_price']:.2f}")
        print(f"Risk-free rate: {r['risk_free_rate']*100:.2f}%")
        print(f"Dividend yield: {r['dividend_yield']*100:.2f}%")
        print(f"Time to maturity: {r['time_to_maturity']:.4f} years")
        print(f"Binomial steps: {r['binomial_steps']}")
        print(f"Pricing model used for sizing: {r['pricing_model'].upper()}")

        print("\n--- PUT (SELL) 95% Moneyness ---")
        print(f"Strike: ${r['put_strike']:.2f}")
        print(f"Implied Vol: {r['put_iv']*100:.2f}%")
        print(f"Contracts: {r['put_contracts']}")
        print(f"  BS price per option:       ${r['put_price_bs']:.4f}")
        print(f"  Binomial price per option: ${r['put_price_binom']:.4f}")
        print(f"  Premium collected ({r['pricing_model'].upper()}): ${r['premium_collected']:,.2f}")

        print("\n--- CALL (BUY) 105% Moneyness ---")
        print(f"Strike: ${r['call_strike']:.2f}")
        print(f"Implied Vol: {r['call_iv']*100:.2f}%")
        print(f"Contracts bought ({r['pricing_model'].upper()} sizing): {r['call_contracts']}")
        print(f"  BS price per option:       ${r['call_price_bs']:.4f}")
        print(f"  Binomial price per option: ${r['call_price_binom']:.4f}")
        print(f"  Premium paid ({r['pricing_model'].upper()}): ${r['premium_paid']:,.2f}")

        print("\n=== NET PREMIUM (based on chosen model) ===")
        print(f"Net Premium: ${r['net_premium']:,.2f}")

        print("\n=== MODEL CHECK ===")
        print(
            "Put:  BS vs Binomial =",
            round(r['put_price_bs'], 4), "vs", round(r['put_price_binom'], 4)
        )
        print(
            "Call: BS vs Binomial =",
            round(r['call_price_bs'], 4), "vs", round(r['call_price_binom'], 4)
        )


In [7]:
def risk_reversal_greeks(
    n_put,
    n_call,
    put_delta,
    call_delta,
    put_gamma,
    call_gamma,
    put_vega,
    call_vega,
    put_theta,
    call_theta,
    put_rho,
    call_rho,
    contract_size=100,
):
    """
    Compute portfolio Greeks for a risk reversal.

    n_put, n_call : number of contracts (positive = long, negative = short)
    Greeks are per-option (not per-contract).
    contract_size : usually 100 for US equity options.
    """

    # Each Greek = sum(contracts * contract_size * per-option greek)
    delta = contract_size * (n_call * call_delta + n_put * put_delta)
    gamma = contract_size * (n_call * call_gamma + n_put * put_gamma)
    vega  = contract_size * (n_call * call_vega  + n_put * put_vega)
    theta = contract_size * (n_call * call_theta + n_put * put_theta)
    rho   = contract_size * (n_call * call_rho   + n_put * put_rho)

    return {
        "delta": delta,
        "gamma": gamma,
        "vega": vega,
        "theta": theta,
        "rho": rho,
    }

In [12]:
five_percent_otm_options_path = '../data/Option/eod_option_5%_OTM.csv'
df = pd.read_csv(five_percent_otm_options_path)

call = df[df["call_put"] == "C"]
spot_price = call["underlying_price"].iloc[0]

call_iv = call["iv"].iloc[0]
call_strike = call["price_strike"].iloc[0]
call_delta = call["delta"].iloc[0]
call_gamma = call["gamma"].iloc[0]
call_vega = call["vega"].iloc[0]
call_theta = call["theta"].iloc[0]
call_rho = call["rho"].iloc[0]

put = df[df["call_put"] == "P"]
put_iv = put["iv"].iloc[0]
put_strike = put["price_strike"].iloc[0]
put_delta = put["delta"].iloc[0]
put_gamma = put["gamma"].iloc[0]
put_vega = put["vega"].iloc[0]
put_theta = put["theta"].iloc[0]
put_rho = put["rho"].iloc[0]

print(f"Spot Price: {spot_price}")

print("\n--- CALL ---")
print(f"Call IV: {call_iv}")
print(f"Call Strike: {call_strike}")
print(f"Call Delta: {call_delta}")
print(f"Call Gamma: {call_gamma}")
print(f"Call Vega: {call_vega}")
print(f"Call Theta: {call_theta}")
print(f"Call Rho: {call_rho}")

print("\n--- PUT ---")
print(f"Put IV: {put_iv}")
print(f"Put Strike: {put_strike}")
print(f"Put Delta: {put_delta}")
print(f"Put Gamma: {put_gamma}")
print(f"Put Vega: {put_vega}")
print(f"Put Theta: {put_theta}")
print(f"Put Rho: {put_rho}")

Spot Price: 543.37

--- CALL ---
Call IV: 0.210945
Call Strike: 571.0
Call Delta: 0.228211
Call Gamma: 0.009197
Call Vega: 0.470783
Call Theta: -0.176452
Call Rho: 0.098448

--- PUT ---
Put IV: 0.323284
Put Strike: 516.0
Put Delta: -0.264147
Put Gamma: 0.006544
Put Vega: 0.51151
Put Theta: -0.262369
Put Rho: -0.111529


In [5]:
inputs = TradeInputs(
    spot_price=spot_price,
    risk_free_rate=0.045,
    dividend_yield=0.013,
    time_to_maturity=1/12,

    put_strike=put_strike,
    put_contracts=1000,
    put_iv=put_iv,

    call_strike=call_strike,
    call_iv=call_iv,

    binomial_steps=1000,
    american=True,
    pricing_model="binomial"
)

analyzer = BullishRiskReversalTradeAnalyzer(inputs=inputs)
analyzer.print_report()      # same text output as your script
results = analyzer.compute() # or use the dict programmatically


=== TRADE SETUP ===
Spot Price: $543.37
Risk-free rate: 4.50%
Dividend yield: 1.30%
Time to maturity: 0.0833 years
Binomial steps: 1000
Pricing model used for sizing: BINOMIAL

--- PUT (SELL) 95% Moneyness ---
Strike: $516.00
Implied Vol: 32.33%
Contracts: 1000
  BS price per option:       $8.5507
  Binomial price per option: $8.5851
  Premium collected (BINOMIAL): $858,509.58

--- CALL (BUY) 105% Moneyness ---
Strike: $571.00
Implied Vol: 21.09%
Contracts bought (BINOMIAL sizing): 2006
  BS price per option:       $4.2805
  Binomial price per option: $4.2796
  Premium paid (BINOMIAL): $858,482.34

=== NET PREMIUM (based on chosen model) ===
Net Premium: $27.25

=== MODEL CHECK ===
Put:  BS vs Binomial = 8.5507 vs 8.5851
Call: BS vs Binomial = 4.2805 vs 4.2796


In [6]:
results

{'spot_price': np.float64(543.37),
 'risk_free_rate': 0.045,
 'dividend_yield': 0.013,
 'time_to_maturity': 0.08333333333333333,
 'binomial_steps': 1000,
 'pricing_model': 'binomial',
 'put_strike': np.float64(516.0),
 'put_iv': np.float64(0.323284),
 'put_contracts': 1000,
 'put_price_bs': np.float64(8.55074710508643),
 'put_price_binom': np.float64(8.58509583269684),
 'call_strike': np.float64(571.0),
 'call_iv': np.float64(0.210945),
 'call_price_bs': np.float64(4.280530819123868),
 'call_price_binom': np.float64(4.279572966227173),
 'premium_collected': np.float64(858509.5832696841),
 'call_contracts': 2006,
 'premium_paid': np.float64(858482.3370251708),
 'net_premium': np.float64(27.246244513313286)}

In [19]:
risk_reversal_greeks(
    results['put_contracts'],
    results['call_contracts'],
    put_delta,
    call_delta,
    put_gamma,
    call_gamma,
    put_vega,
    call_vega,
    put_theta,
    call_theta,
    put_rho,
    call_rho,
    contract_size=100,
)

{'delta': np.float64(19364.426599999995),
 'gamma': np.float64(2499.3182),
 'vega': np.float64(145590.06980000003),
 'theta': np.float64(-61633.171200000004),
 'rho': np.float64(8595.7688)}

Following the historic rally, the trader initiated a bullish risk-reversal position—short 95% puts and long 105% calls—and the resulting Greeks reveal an aggressively leveraged, high-convexity bullish profile that carries significant risk.

### **Key Observations**

**Delta: +19,364.43**
The portfolio exhibits an extremely strong positive delta, implying substantial bullish exposure. This level of delta suggests that the long call position, funded by premium collected from the short puts, is very large—likely several thousand contracts. As a result, the portfolio gains roughly **$19,364** for every 1-point increase in SPY. The trader is essentially holding a large synthetic long equity position.

**Gamma: +2,499.32**
The positive gamma indicates that the portfolio’s delta will increase further as SPY rises, providing upside acceleration. This convexity is characteristic of being net long options, meaning the long calls overwhelmingly dominate the profile. However, on the downside, although gamma reduces the delta as SPY falls, the trader still carries substantial exposure through the large short put position.

**Vega: +145,590.07**
The portfolio is massively long volatility. A 1-point drop in implied volatility results in approximately **$145,590** of losses. Entering the trade after a major rally is particularly dangerous, as implied volatility typically collapses following such events. This creates a scenario where the trader is exposed to significant mark-to-market losses even if the underlying moves upward.

**Theta: –61,633.17**
Time decay is severe: the trader loses over **$61,000 per day** from theta alone. This reflects the net-long-premium structure—long expensive OTM calls and short relatively cheap puts. With only a month until expiration, this decay will accelerate, requiring the underlying to move sharply and quickly to offset the decay.

**Rho: +8,595.77**
The position has moderately positive rho, benefiting from rising interest rates. While this is far less impactful than the delta, vega, and theta exposures, it still contributes marginally to the overall profile.

---

### **Critical Risks Identified**

1. **Volatility Crush Risk**
   Because the position is so heavily long volatility, any post-rally volatility collapse could cause substantial losses. This is especially concerning given the historical tendency for implied volatility to compress sharply after large upward moves.

2. **Severe Theta Bleed**
   The portfolio is losing more than $61,000 per day due to time decay. Given the short time to expiration, the trader faces rapidly accelerating theta losses unless SPY rallies hard and fast.

3. **Timing Risk**
   The trader entered after a massive rally—typically the worst time to purchase upside calls:

   * Calls are inflated due to demand and elevated skew dynamics.
   * Puts have already seen their fear premium eroded.
   * Markets often consolidate or retrace after major rallies.

4. **Asymmetric Downside Risk**
   While the upside is theoretically unlimited, the short puts introduce substantial downside risk. Any sharp reversal could lead to significant losses well before the calls regain value.

---

### **Conclusion**

Overall, the trader has constructed an aggressively bullish position at a moment when the risk–reward profile is unfavorable. The extreme vega and theta exposure means that without a **rapid and continued upside breakout**, the position is vulnerable to large mark-to-market losses. Even a benign sideways market or a modest volatility contraction could turn the trade sharply negative.
